<a href="https://colab.research.google.com/github/h4ck4l1/datasets/blob/main/NLP_with_RNN_and_Attention/NMT_with_attention_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os,warnings,sys
if "google.colab" in sys.modules:
    !pip3 install -q -U "tensorflow-text==2.13.0"
    !pip3 install -q -U einops
warnings.filterwarnings("ignore")
from IPython.display import clear_output
os.environ["TF_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from tensorflow import keras
import tensorflow_text as text
import typing
from zipfile import ZipFile
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import einops
pio.templates.default = "plotly_dark"
import numpy as np
np.set_printoptions(precision=2)
tf.get_logger().setLevel("ERROR")
%xmode Minimal
if "google.colab" in sys.modules:
    clear_output()

In [2]:

class ShapeCheck():

    def __init__(self):

        self.shapes = {}

    def __call__(self,tensor,names,broadcast=False):

        parsed = einops.parse_shape(tensor,names)

        for name,new_dim in parsed.items():

            old_dim = self.shapes.get(name,None)

            if broadcast and (new_dim == 1):
                continue

            if old_dim is None:

                self.shapes[name] = new_dim
                continue

            if new_dim != old_dim:

                raise ValueError(f"shape mismatch for dimension: '{name}' found: {new_dim} expected: {old_dim}")

In [3]:
url = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"

In [4]:
file_path = keras.utils.get_file(fname="spa-eng.zip",origin=url,extract=True)

with ZipFile(file_path,"r") as f:

    f.extractall("spa-eng")

with open("spa-eng/spa-eng/spa.txt","r") as f:

    total_text = f.read()
    total_text = [line.split("\t") for line in total_text.splitlines()]
    en_text,es_text = zip(*total_text)

2638744/2638744 [==============================] - 1s 0us/step


In [5]:
en_text[-1]

'If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.'

In [6]:
es_text[-1]

'Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado.'

In [7]:
en_array = np.array(en_text)
es_array = np.array(es_text)

In [8]:
is_train = np.random.uniform(size=(len(en_array),)) < 0.8

raw_train = (
    tf.data.Dataset
    .from_tensor_slices((en_array[is_train],es_array[is_train]))
    .shuffle(len(en_text))
    .batch(64)
)
raw_valid = (
    tf.data.Dataset
    .from_tensor_slices((en_array[~is_train],es_array[~is_train]))
    .shuffle(len(en_text))
    .batch(64)
)

In [9]:
for en,es in raw_train.take(1):
    print(en[:4])
    print("translates to latin as ")
    print(es[:4])

tf.Tensor(
[b'How many close friends do you have?' b'Tom loved his children.'
 b"They're right." b'Why are you so insecure?'], shape=(4,), dtype=string)
translates to latin as 
tf.Tensor(
[b'\xc2\xbfCu\xc3\xa1ntos buenos amigos tienes?'
 b'Tom quer\xc3\xada a sus hijos.' b'Tienen raz\xc3\xb3n.'
 b'\xc2\xbfPor qu\xc3\xa9 sos tan insegura?'], shape=(4,), dtype=string)


# Standardize Text

In [10]:
es_text[:10]

('Ve.',
 'Vete.',
 'Vaya.',
 'Váyase.',
 'Hola.',
 '¡Corre!',
 'Corred.',
 '¿Quién?',
 '¡Fuego!',
 '¡Incendio!')

In [11]:
tf.constant(es_text[:10]) # converting to tensor

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Ve.', b'Vete.', b'Vaya.', b'V\xc3\xa1yase.', b'Hola.',
       b'\xc2\xa1Corre!', b'Corred.', b'\xc2\xbfQui\xc3\xa9n?',
       b'\xc2\xa1Fuego!', b'\xc2\xa1Incendio!'], dtype=object)>

In [12]:
temp_text = text.normalize_utf8(es_text[:10],"NFKD") # Normalizing text so that it can be used in operations
temp_text

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Ve.', b'Vete.', b'Vaya.', b'Va\xcc\x81yase.', b'Hola.',
       b'\xc2\xa1Corre!', b'Corred.', b'\xc2\xbfQuie\xcc\x81n?',
       b'\xc2\xa1Fuego!', b'\xc2\xa1Incendio!'], dtype=object)>

In [13]:
temp_text_1 = tf.strings.lower(temp_text) # Lower casing all the characters
temp_text_1

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b've.', b'vete.', b'vaya.', b'va\xcc\x81yase.', b'hola.',
       b'\xc2\xa1corre!', b'corred.', b'\xc2\xbfquie\xcc\x81n?',
       b'\xc2\xa1fuego!', b'\xc2\xa1incendio!'], dtype=object)>

In [14]:
temp_text_2 = tf.strings.regex_replace(temp_text_1,"[^ a-z.?!,¿]","")  # [^ ...] means exclude..so excluding all the a-z and rest
temp_text_2                                                            # and replacing with noting

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b've.', b'vete.', b'vaya.', b'vayase.', b'hola.', b'corre!',
       b'corred.', b'\xc2\xbfquien?', b'fuego!', b'incendio!'],
      dtype=object)>

In [15]:
temp_text_3 = tf.strings.regex_replace(temp_text_2,"[.¡¿,?!]",r' \0 ') # Placing a null character[raw_string : r'']
temp_text_3                                                            # before and after every punctuation

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b've . ', b'vete . ', b'vaya . ', b'vayase . ', b'hola . ',
       b'corre ! ', b'corred . ', b' \xc2\xbf quien ? ', b'fuego ! ',
       b'incendio ! '], dtype=object)>

In [16]:
temp_text_4= tf.strings.strip(temp_text_3) # stripping any extra spaces
temp_text_4

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b've .', b'vete .', b'vaya .', b'vayase .', b'hola .', b'corre !',
       b'corred .', b'\xc2\xbf quien ?', b'fuego !', b'incendio !'],
      dtype=object)>

In [17]:
temp_text_5 = tf.strings.join(['[startofsequence]',temp_text_4,'[endofsequence]'],separator=" ")
temp_text_5

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'[startofsequence] ve . [endofsequence]',
       b'[startofsequence] vete . [endofsequence]',
       b'[startofsequence] vaya . [endofsequence]',
       b'[startofsequence] vayase . [endofsequence]',
       b'[startofsequence] hola . [endofsequence]',
       b'[startofsequence] corre ! [endofsequence]',
       b'[startofsequence] corred . [endofsequence]',
       b'[startofsequence] \xc2\xbf quien ? [endofsequence]',
       b'[startofsequence] fuego ! [endofsequence]',
       b'[startofsequence] incendio ! [endofsequence]'], dtype=object)>

In [18]:
def text_preprocessor(input_text):

    input_text = text.normalize_utf8(input_text,"NFKD")
    input_text = tf.strings.lower(input_text)
    input_text = tf.strings.regex_replace(input_text,"[^ a-z?.!¿¡,]","")
    input_text = tf.strings.regex_replace(input_text,"[?.!¿¡,]",r" \0 ")
    input_text = tf.strings.strip(input_text)
    input_text = tf.strings.join(["[startofsequence]",input_text,"[endofsequence]"],separator=" ")
    return input_text

# Text Vectorization of En and Es

In [19]:
vocab_size = 5000

en_vec_layer = keras.layers.TextVectorization(vocab_size,standardize=text_preprocessor,ragged=True)
en_vec_layer.adapt(raw_train.map(lambda en,es:en))
es_vec_layer = keras.layers.TextVectorization(vocab_size,standardize=text_preprocessor,ragged=True)
es_vec_layer.adapt(raw_train.map(lambda en,es:es))

In [20]:
print(en_vec_layer.get_vocabulary()[:10])
print(es_vec_layer.get_vocabulary()[:10])

['', '[UNK]', '[startofsequence]', '[endofsequence]', '.', 'the', 'i', 'to', 'you', 'tom']
['', '[UNK]', '[startofsequence]', '[endofsequence]', '.', 'que', 'de', 'el', 'a', 'no']


In [21]:
for english_text,en_vectorized_out in zip(en.numpy()[:4],en_vec_layer(en[:4])):
    print(english_text,"---->",en_vectorized_out)

b'How many close friends do you have?' ----> tf.Tensor([  2  58 121 518 218  20   8  23  11   3], shape=(10,), dtype=int64)
b'Tom loved his children.' ----> tf.Tensor([  2   9 630  30 213   4   3], shape=(7,), dtype=int64)
b"They're right." ----> tf.Tensor([  2 288 151   4   3], shape=(5,), dtype=int64)
b'Why are you so insecure?' ----> tf.Tensor([   2   80   28    8   90 3065   11    3], shape=(8,), dtype=int64)


In [22]:
for latin_text,es_vectorized_out in zip(es.numpy()[:4],es_vec_layer(es[:4])):
    print(latin_text,"---->",es_vectorized_out)

b'\xc2\xbfCu\xc3\xa1ntos buenos amigos tienes?' ----> tf.Tensor([  2  13 454 896 192  92  12   3], shape=(8,), dtype=int64)
b'Tom quer\xc3\xada a sus hijos.' ----> tf.Tensor([  2  10 124   8  88 375   4   3], shape=(8,), dtype=int64)
b'Tienen raz\xc3\xb3n.' ----> tf.Tensor([  2 265 385   4   3], shape=(5,), dtype=int64)
b'\xc2\xbfPor qu\xc3\xa9 sos tan insegura?' ----> tf.Tensor([  2  13  21   5 507  64   1  12   3], shape=(9,), dtype=int64)


In [23]:
en_vocab = np.array(en_vec_layer.get_vocabulary())
es_vocab = np.array(es_vec_layer.get_vocabulary())

In [24]:
print(" ".join(en_vocab[en_vectorized_out.numpy()]))
print(" ".join(es_vocab[es_vectorized_out.numpy()]))

[startofsequence] why are you so insecure ? [endofsequence]
[startofsequence] ¿ por que sos tan [UNK] ? [endofsequence]


In [25]:
en_vec_out = en_vec_layer(en)
es_vec_out = es_vec_layer(es)

In [26]:
fig = make_subplots(cols=2,subplot_titles=["Unmasked","Masked"])
fig.add_trace(go.Heatmap(z=en_vec_out.to_tensor().numpy()),row=1,col=1)
fig.add_trace(go.Heatmap(z=np.array((en_vec_out.to_tensor() != 0).numpy(),dtype=np.int32)),row=1,col=2)
fig.show()

In [27]:
def preprocess_dataset(en,es):
    X_train = en_vec_layer(en).to_tensor()
    X_dec = es_vec_layer(es)
    X_dec_train = X_dec[:,:-1].to_tensor()
    y_train = X_dec[:,1:].to_tensor()

    return (X_train,X_dec_train),y_train

In [28]:
train_ds = raw_train.map(preprocess_dataset,tf.data.AUTOTUNE)
valid_ds = raw_valid.map(preprocess_dataset,tf.data.AUTOTUNE)

In [29]:
for (en_in,es_in),es_out in train_ds.take(1):
    print(en_in.shape)
    print(es_in.shape)
    print(es_out.shape)

(64, 19)
(64, 17)
(64, 17)


# Encoder Class

- Embedding Layer
- GRU/LSTM Layer

In [30]:
vocab_size = len(en_vec_layer.get_vocabulary())
vocab_size

5000

In [31]:
embed_size = 256

In [32]:
encoder_embed_layer = keras.layers.Embedding(vocab_size,embed_size,mask_zero=True)
encoder = keras.layers.Bidirectional(
    keras.layers.LSTM(256,return_sequences=True,recurrent_initializer="glorot_uniform"),
    merge_mode="sum"
)

In [33]:
shape_checker = ShapeCheck()
shape_checker(en_in,"batch s")

In [34]:
shape_checker.shapes

{'batch': 64, 's': 19}

In [35]:
enc_embed_output = encoder_embed_layer(en_in)
enc_embed_output.shape

TensorShape([64, 19, 256])

In [36]:
shape_checker(enc_embed_output,"batch s units")
shape_checker.shapes

{'batch': 64, 's': 19, 'units': 256}

In [37]:
encoder_outputs = encoder(enc_embed_output)
encoder_outputs.shape

TensorShape([64, 19, 256])

In [38]:
shape_checker(encoder_outputs,"batch s units")

In [39]:
mha = keras.layers.MultiHeadAttention(num_heads=1,key_dim=256)

In [40]:
print(en_in.shape)
print(encoder_outputs.shape)

(64, 19)
(64, 19, 256)


In [41]:
len(en_vec_layer.get_vocabulary())

5000

In [42]:
class Encoder(keras.Model):

    def __init__(self,units=256,vec_layer=en_vec_layer,**kwargs):

        super(Encoder,self).__init__(**kwargs)
        self.vec_layer = vec_layer
        self.embed = keras.layers.Embedding(vec_layer.vocabulary_size(),units,mask_zero=True)
        self.Rnn = keras.layers.Bidirectional(
            layer=keras.layers.LSTM(units,return_sequences=True,recurrent_initializer="glorot_uniform"),
            merge_mode="sum"
        )

    def call(self,inputs):

        shape_checker = ShapeCheck()
        shape_checker(inputs,"batch s")
        z = self.embed(inputs)
        shape_checker(z,"batch s units")
        z = self.Rnn(z)
        shape_checker(z,"batch s units")
        return z


# CrossAttention

In [43]:
decoder_embed_layer = keras.layers.Embedding(es_vec_layer.vocabulary_size(),256,mask_zero=True)
decoder_embed_out = decoder_embed_layer(es_in)
decoder_embed_out.shape

TensorShape([64, 17, 256])

In [44]:
mha = keras.layers.MultiHeadAttention(num_heads=1,key_dim=256)
attention_output,attention_scores = mha(query=decoder_embed_out,value=encoder_outputs,return_attention_scores=True)
print(attention_output.shape)
print(attention_scores.shape)

(64, 17, 256)
(64, 1, 17, 19)


In [45]:
shape_checker = ShapeCheck()
shape_checker(decoder_embed_out,"batch t units")
shape_checker(encoder_outputs,"batch s units")

In [46]:
attention_scores = tf.reduce_mean(attention_scores,axis=1)
attention_scores.shape

TensorShape([64, 17, 19])

In [47]:
adding_layer = keras.layers.Add()
add_out = adding_layer([decoder_embed_out,attention_output])
add_out.shape

TensorShape([64, 17, 256])

In [48]:
layer_norm = keras.layers.LayerNormalization()
layer_out = layer_norm(add_out)
layer_out.shape

TensorShape([64, 17, 256])

In [49]:
class CrossAttention(keras.layers.Layer):

    def __init__(self,units=256,**kwargs):

        super(CrossAttention,self).__init__(**kwargs)
        self.mha = keras.layers.MultiHeadAttention(num_heads=1,key_dim=units)
        self.add = keras.layers.Add()
        self.layer_norm = keras.layers.LayerNormalization()

    def call(self,decoder_out,encoder_out):

        shape_checker = ShapeCheck()
        shape_checker(decoder_out,"batch t units")
        shape_checker(encoder_out,"batch s units")

        attention_output,attention_scores = self.mha(query=decoder_out,value=encoder_out,return_attention_scores=True)
        shape_checker(attention_output,"batch t units")
        shape_checker(attention_scores,"batch heads t s")

        add_and_layer_norm = self.layer_norm(self.add([decoder_out,attention_output]))
        self.attention_scores = tf.reduce_mean(attention_scores,axis=1)

        return add_and_layer_norm

In [50]:
attention_layer = CrossAttention()

attention_out = attention_layer(decoder_embed_out,encoder_outputs)
attention_out.shape

TensorShape([64, 17, 256])

In [51]:
attention_layer.attention_scores.shape

TensorShape([64, 17, 19])

In [52]:
np.sum(attention_layer.attention_scores,axis=-1)[:5,:]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.]], dtype=float32)

In [53]:
fig = make_subplots(cols=2,subplot_titles=["Attention Output","Masked Output"])
fig.add_trace(go.Heatmap(z=attention_layer.attention_scores[:,0,:]),row=1,col=1)
fig.add_trace(go.Heatmap(z=np.array((en_vec_out.to_tensor() != 0).numpy(),dtype=np.int32)),row=1,col=2)
fig.show()

# Decoder

In [53]:
class Decoder(keras.layers.Layer):

    def __init__(self,units=256,vec_layer=es_vec_layer,**kwargs):

        super(Decoder,self).__init__(**kwargs)

        '''Preprocessing Section'''
        self.vec_layer = vec_layer
        self.vocab_size = vec_layer.vocabulary_size()
        self.word_to_id = keras.layers.StringLookup(
            vocabulary=vec_layer.get_vocabulary(),
            oov_token="[UNK]",
            mask_token=""
        )
        self.id_to_word = keras.layers.StringLookup(
            vocabulary=vec_layer.get_vocabulary(),
            oov_token="[UNK]",
            mask_token="",
            invert=True
        )
        self.start_token = self.word_to_id('[startofsequence]')
        self.end_token = self.word_to_id('[endofsequence]')
        self.units = units
        '''Model Layers section'''
        self.es_embed = keras.layers.Embedding(vec_layer.vocabulary_size(),units,mask_zero=True)
        self.decoder = keras.layers.LSTM(units,return_sequences=True,return_state=True,recurrent_initializer="glorot_uniform")
        self.attention = CrossAttention()
        self.out = keras.layers.Dense(vec_layer.vocabulary_size())


    def call(self,inputs):
        pass